## Evaluating full test set performance

In [1]:
import torch
#from torch.autograd import Variable
from capsnet import CapsNet
import utils
USE_GPU = False 
datasets = ['MNIST', 'FashionMNIST']

### Load model and weights from past training

In [2]:
model = CapsNet(n_conv_channel=256,
                n_primary_caps=8,
                primary_cap_size=1152,
                output_unit_size=16,
                n_routing_iter=3)

In [3]:
def test(epoch, model, dataloader):
    accs = []
    losses = []
    model.eval()

    for i, (X, y) in enumerate(dataloader):
        target = utils.one_hot(y, model.final_caps.n_unit)

        #X, target = Variable(X), Variable(target)
        if USE_GPU:
            X, target = X.cuda(), target.cuda()

        y_hat = model(X)
        loss = model.loss(y_hat, target)

        preds = model.capsule_prediction(y_hat)
        acc = utils.categorical_accuracy(y.float(), preds.cpu().data)
        
        accs.append(acc)
        losses.append(loss.data.item())        
    return accs, losses

### Test on MNIST

In [4]:
model.load_state_dict(torch.load('./checkpoints/mnist_capsnet_50_epoch.pth', map_location='cpu'))
model.eval()
model = model.cuda() if USE_GPU else model
input_data = {'data':datasets[0], 'download':False}
trainloader, testloader = utils.mnist_dataloaders('./data', use_cuda=USE_GPU, **input_data)

MNIST


In [5]:
accs, losses = test(1, model, testloader)

F:\INM705-2022\Labs\capsulenet-master\capsulenet-master\capsule_layer.py:88: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  c_ij = F.softmax(b_ij)
F:\INM705-2022\Labs\capsulenet-master\capsulenet-master\capsnet.py:83: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  dim=2, keepdim=True)).view(-1, 10)


In [6]:
accs, losses = torch.Tensor(accs), torch.Tensor(losses)

### Average accuracy and loss over the entire test set

In [7]:
accs.mean()

tensor(0.9901)

In [8]:
print('Accuracy: {:.3f}% Loss: {:.5f}'.format(100*accs.mean(), losses.mean()))

Accuracy: 99.011% Loss: 0.03029


### Test on Fashion MNIST

In [9]:
model = CapsNet(n_conv_channel=256,
                n_primary_caps=8,
                primary_cap_size=1152,
                output_unit_size=16,
                n_routing_iter=3)
model.load_state_dict(torch.load('./checkpoints/fmnist_capsnet_50_epoch.pth', map_location='cpu'))
model.eval()
model = model.cuda() if USE_GPU else model
input_data = {'data':datasets[1], 'download':False}
trainloader, testloader = utils.mnist_dataloaders('./data', use_cuda=USE_GPU, **input_data)

FashionMNIST
FashionMNIST


In [10]:
accs, losses = test(1, model, testloader)
accs, losses = torch.Tensor(accs), torch.Tensor(losses)
print('Accuracy: {:.3f}% Loss: {:.5f}'.format(100*accs.mean(), losses.mean()))

Accuracy: 88.904% Loss: 0.16416
